### Phase 1: Import Modules

In [137]:
# Import boto3 and json modules
# json module is required and every aws response is given in json format.

import boto3
import json

In [138]:
# Create ec2 client

client = boto3.client('ec2')

# check this url to find all the methods that can be used with ec2 client
# https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/ec2.html

### Phase 2: Get the list of security groups with ssh rule open to the world

In [139]:
# scan all security groups in our aws account
sg_rules = client.describe_security_group_rules().get('SecurityGroupRules')

In [140]:
# get a list of all security group id
sg_list = [] # will possibly contain duplicate values

for item in sg_rules:
    sg = item.get('GroupId')
    sg_list.append(sg)
sg_id_list = set(sg_list) # Get a list of unique values
sg_id_list

{'sg-0161af317ef49e2ee',
 'sg-03352996fd1aa33e4',
 'sg-0346cac00c1ca954b',
 'sg-088bf9b8fc8cb6fbf',
 'sg-08bb57f369cb9f105',
 'sg-0ab033979b01aaeab',
 'sg-0b52c68e9eb84e6e2',
 'sg-0b874162485f503f9'}

In [141]:
# get a list of all security group id with ssh rule open to the world

sg_with_ssh_open_to_world = []

for item in sg_rules:
    sg = item.get('GroupId')
    if item.get('FromPort')==22 and item.get('CidrIpv4')=='0.0.0.0/0':
        sg_with_ssh_open_to_world.append(sg)

sg_with_ssh_open_to_world

['sg-0ab033979b01aaeab',
 'sg-0b52c68e9eb84e6e2',
 'sg-03352996fd1aa33e4',
 'sg-08bb57f369cb9f105',
 'sg-0b874162485f503f9']

### Phase 3: Get the list of ec2 instance in active state that have the sg obtained from phase 2

In [142]:
# Get the list of all active ec2 instances in our account

resource = client.describe_instances().get('Reservations')  # Here, we filter to get only the list of the key "Reservation"

ec2_instances = [] # Here, we will filter the reservation list and get the sub-list with key "Instances"

for item in resource:
    instance = item.get('Instances')
    ec2_instances.append(instance)

ec2_instances

[[{'AmiLaunchIndex': 0,
   'ImageId': 'ami-0c101f26f147fa7fd',
   'InstanceId': 'i-037747891fdecc6ca',
   'InstanceType': 't2.micro',
   'LaunchTime': datetime.datetime(2024, 4, 1, 8, 43, 24, tzinfo=tzutc()),
   'Monitoring': {'State': 'disabled'},
   'Placement': {'AvailabilityZone': 'us-east-1a',
    'GroupName': '',
    'Tenancy': 'default'},
   'PrivateDnsName': '',
   'ProductCodes': [],
   'PublicDnsName': '',
   'State': {'Code': 48, 'Name': 'terminated'},
   'StateTransitionReason': 'User initiated (2024-04-01 08:47:29 GMT)',
   'Architecture': 'x86_64',
   'BlockDeviceMappings': [],
   'ClientToken': '9f45a969-2096-4e67-bc65-d81bb5629d90',
   'EbsOptimized': False,
   'EnaSupport': True,
   'Hypervisor': 'xen',
   'NetworkInterfaces': [],
   'RootDeviceName': '/dev/xvda',
   'RootDeviceType': 'ebs',
   'SecurityGroups': [],
   'StateReason': {'Code': 'Client.UserInitiatedShutdown',
    'Message': 'Client.UserInitiatedShutdown: User initiated shutdown'},
   'Tags': [{'Key': 'Na

In [143]:
# We extract all instance with state in either "running" or "stopped"
running_ec2_instances = []

for item1 in ec2_instances:
    for item2 in item1:
        v = item2.get('State').get('Name')
        if v not in ('running', 'stopped'):
            pass
        else:
            running_ec2_instances.append(item2)

running_ec2_instances

[]

In [144]:
running_ec2_instances_id = [] # all ec2 instance id with security ssh rule open to the world
count = 0 # To enable us count the number of instances that will be terminated

for item1 in running_ec2_instances:  # we loop into the securityGroups list to get the security group id
    id = item1.get('InstanceId')
    sg = item1.get('SecurityGroups')
    for item2 in sg:
        sg_id = item2.get('GroupId') # we loop into the securityGroups list to get the security group id
        if sg_id in sg_with_ssh_open_to_world:
            running_ec2_instances_id.append(id)
            count = count+1
running_ec2_instances_id = list(set(running_ec2_instances_id)) # remove all duplicates using the "set" function, then converting it to a list
print(running_ec2_instances_id)
print(count,"ec2 instance found.")

[]
0 ec2 instance found.


### Phase 4: Take remedy actions

In [145]:
# We will terminate all ec2 instance id obtained from phase 3.

try:
    response = client.terminate_instances(InstanceIds = running_ec2_instances_id)
    print(count, "ec2 instances have been successfully terminated.")
except:
    print('There is no running ec2 intance with security group ssh rule opened to the world')

There is no running ec2 intance with security group ssh rule opened to the world
